# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [3]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()


,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [105]:
# Create our features
X = df.drop(columns = ['loan_status'])
X = pd.get_dummies(X, columns = ['homeowner'])

# Create our target
y = df['loan_status'].to_frame()

In [107]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [108]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [109]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

display(y_train.value_counts())

loan_status
low_risk       50224
high_risk       1725
dtype: int64

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [110]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [111]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_train_info = scaler.fit(X_train)

In [112]:
# Scale the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


# Simple Logistic Regression

In [113]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [114]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.98703756767017

In [115]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  759,    16],
       [  131, 24681]])

In [117]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.98      0.99      0.91      0.99      0.97       775
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     24812

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     25587



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [121]:
from imblearn.over_sampling import RandomOverSampler
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=0)
X_resampled_train, y_resampled_train = ros.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_train.loan_status)

Counter({'low_risk': 50224, 'high_risk': 1725})

In [122]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_train, y_resampled_train)

LogisticRegression(random_state=1)

In [123]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9950619367319663

In [124]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  772,     3],
       [  149, 24663]])

In [125]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       775
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     24812

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     25587



### SMOTE Oversampling

In [128]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_smote_train, y_smote_train = oversample.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_smote_train.loan_status)

Counter({'low_risk': 50224, 'high_risk': 50224})

In [129]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_smote_train, y_smote_train)

LogisticRegression(random_state=1)

In [130]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9950619367319663

In [131]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  772,     3],
       [  149, 24663]])

In [132]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       775
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     24812

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     25587



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [133]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids 
cc = ClusterCentroids(random_state=42)
X_under_train, y_under_train = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_under_train.loan_status)

Counter({'high_risk': 1725, 'low_risk': 1725})

In [134]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_under_train, y_under_train)

LogisticRegression(random_state=1)

In [135]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9928238677434956

In [136]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  768,     7],
       [  132, 24680]])

In [137]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.99      0.99      0.92      0.99      0.99       775
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     24812

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     25587



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [138]:
from imblearn.combine import SMOTEENN 
# Resample the training data with SMOTEENN
SMOTEENN = SMOTEENN()
X_smoteenn_train, y_smoteenn_train = SMOTEENN.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_smoteenn_train.loan_status)

Counter({'high_risk': 49448, 'low_risk': 49881})

In [139]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_smoteenn_train, y_smoteenn_train)

LogisticRegression(random_state=1)

In [140]:
# Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9951022398111216

In [141]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  772,     3],
       [  147, 24665]])

In [142]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       775
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     24812

avg / total       1.00      0.99      1.00      0.99      1.00      0.99     25587



In [28]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler

# Classification

def logistic_regression(X_train, y_train, X_test, y_test, X):
    from sklearn.linear_model import LogisticRegression
    model = LogisticRegression(solver='lbfgs', random_state=1)
    model.fit(X_train, y_train)
    
    from sklearn.metrics import balanced_accuracy_score
    y_pred = model.predict(X_test)
    print('')
    print(f"Balanced Accuracy Score {(balanced_accuracy_score(y_test, y_pred))}")
    print('')
    # Display the confusion matrix
    from sklearn.metrics import confusion_matrix
    display(confusion_matrix(y_test, y_pred))
    print('')
    # Print the imbalanced classification report
    from imblearn.metrics import classification_report_imbalanced
    print(classification_report_imbalanced(y_test, y_pred))
    print('')
    display(pd.DataFrame({'Variable': ['intercept'] + list(X.columns), 'Coefficient': list(model.intercept_) + list(list(model.coef_)[0])}).set_index('Variable'))

    
def data_dummifier(data):
    display(data.head())
    display(data.info())
    dependent_variable = str(input("Enter y variable from column"))
    while dependent_variable not in data.columns:
        print(f"Choose from: {data.columns}")
        dependent_variable = str(input("Enter y variable from column"))
        
    dummy_list = []
    dummy_variables = str(input("Enter done to finish dummifying"))
    
    while dummy_variables.lower() != 'done':
        if dummy_variables == dependent_variable:
            print(f"Choose from: {data.columns}, don't choose {dependent_variable}")
            dependent_variable = str(input("Enter y variable from column"))
            
        if dummy_variables not in data.columns:
            print(f"Choose from: {data.columns}, don't choose {dependent_variable}")
            dependent_variable = str(input("Enter y variable from column"))
            
        elif dummy_variables in data.columns:
            dummy_list.append(dummy_variables)
            dummy_variables = str(input("Enter done to finish dummifying"))
            
        else:
            print(f"Choose from: {data.columns}")
            dependent_variable = str(input("Enter y variable from column"))
        
    y = data[dependent_variable].to_frame()
    X = data.drop(columns = [dependent_variable])
    X = pd.get_dummies(X, columns = dummy_list)
    
    datetime_variable = str(input("Enter done to finish date-ifying"))
    datetime_list = []
    while datetime_variable.lower() != 'done':
        if datetime_variable == dependent_variable or datetime_variable in dummy_list:
            print(f"Choose from: {data.columns}")
            dependent_variable = str(input(f"Enter y variable from column, don't choose {dependent_variable} or dummy variables"))
            
        if datetime_variable not in data.columns:
            print(f"Choose from: {data.columns}, don't choose {dependent_variable}")
            dependent_variable = str(input("Enter y variable from column"))
            
        elif datetime_variable in data.columns:
            datetime_list.append(dummy_variables)
            dummy_variables = str(input("Enter done to finish datetiming"))
            
        else:
            print(f"Choose from: {data.columns}")
            dependent_variable = str(input("Enter y variable from column"))
            
    for var in datetime_list:
        X[var] = pd.to_datetime(X[var])
        
    X = pd.get_dummies(X, columns = datetime_list)
    
    return X, y, dependent_variable

def resampler(data, test_split = None, method = None):
    X, y, dependent_variable = data_dummifier(data)
    
    # Train and test split
    from sklearn.model_selection import train_test_split
    if test_split == None:
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

        display(y_train.value_counts())
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, random_state=42)

        display(y_train.value_counts())
    
    if method == None:
        method = str(input("Input resampling method from list [SMOTEENN, ClusterCentroids, SMOTE, RandomOverSampler, None]"))
        while method not in ['SMOTEENN', 'ClusterCentroids', 'SMOTE', 'RandomOverSampler', 'None']:
            method = str(input("Input resampling method from list [SMOTEENN, ClusterCentroids, SMOTE, RandomOverSampler, None]"))

    if method == 'SMOTEENN':
        from imblearn.combine import SMOTEENN 
        # Resample the training data with SMOTEENN
        SMOTEENN = SMOTEENN(random_state=42)
        X_train, y_train = SMOTEENN.fit_resample(X_train, y_train)
        
        # View the count of target classes with Counter
        display(Counter(y_train[dependent_variable]))
        
        logistic_regression(X_train, y_train, X_test, y_test, X)
        
    
    elif method == 'ClusterCentroids':
        from imblearn.under_sampling import ClusterCentroids 
        cc = ClusterCentroids(random_state=42)
        X_train, y_train = cc.fit_resample(X_train, y_train)

        # View the count of target classes with Counter
        display(Counter(y_train[dependent_variable]))
        
        logistic_regression(X_train, y_train, X_test, y_test, X)
        
    elif method == 'SMOTE':
        from imblearn.over_sampling import SMOTE
        cc = SMOTE(random_state=42)
        X_train, y_train = cc.fit_resample(X_train, y_train)

        # View the count of target classes with Counter
        Counter(y_train[dependent_variable])
        
        logistic_regression(X_train, y_train, X_test, y_test, X)
        
    elif method == 'RandomOverSampler':
        from imblearn.over_sampling import RandomOverSampler 
        cc = RandomOverSampler(random_state=42)
        X_train, y_train = cc.fit_resample(X_train, y_train)

        # View the count of target classes with Counter
        display(Counter(y_train[dependent_variable]))
        
        logistic_regression(X_train, y_train, X_test, y_test, X)
        
    elif method == 'None':
        display(Counter(y_train[dependent_variable]))
        logistic_regression(X_train, y_train, X_test, y_test, X)

resampler(df)
    
    

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77536 entries, 0 to 77535
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loan_size         77536 non-null  float64
 1   interest_rate     77536 non-null  float64
 2   homeowner         77536 non-null  object 
 3   borrower_income   77536 non-null  int64  
 4   debt_to_income    77536 non-null  float64
 5   num_of_accounts   77536 non-null  int64  
 6   derogatory_marks  77536 non-null  int64  
 7   total_debt        77536 non-null  int64  
 8   loan_status       77536 non-null  object 
dtypes: float64(3), int64(4), object(2)
memory usage: 5.3+ MB


None

Enter y variable from column loan_status
Enter done to finish dummifying homeowner
Enter done to finish dummifying done
Enter done to finish date-ifying done


loan_status
low_risk       56277
high_risk       1875
dtype: int64

Input resampling method from list [SMOTEENN, ClusterCentroids, SMOTE, RandomOverSampler, None] SMOTE



Balanced Accuracy Score 0.9948279972279972



array([[  622,     3],
       [  104, 18655]])


                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384




,Coefficient
Variable,
intercept,4.404023e-08
loan_size,-2.180291e-03
interest_rate,3.300173e-07
borrower_income,7.624413e-04
debt_to_income,-3.523096e-06
num_of_accounts,6.302525e-05
derogatory_marks,1.304280e-04
total_debt,-5.587642e-04
homeowner_mortgage,2.855886e-04


# Final Questions

1. Which model had the best balanced accuracy score?

   Simple Logistic regression with no sampling

2. Which model had the best recall score?

    All had equal recall scores, however SMOTEENN had higher recall scores for high risks situations, meaning a better model for indicating relatively more important information of higher risk

3. Which model had the best geometric mean score?

    NAIVE, SMOTE, SMOTEENN
